# HW Ranking Data Analysis

### 1. Combining data from multiple files

In [605]:
import glob, os
import numpy as np
import pandas as pd

Concatenating multiple csv files and adding their names to the DataFrame:

In [606]:
all_files = glob.glob("data/*.csv")
li = []
raw_df = pd.concat((pd.read_csv(f).assign(player = f[5:-4]) for f in all_files))

### 2. Decoding and fixing the data

In [607]:
import re

Cleaning up the decoding file, then making a DataFrame out of it:

In [608]:
with open(os.path.join('data', 'DECODING.txt'), 'r') as f:
    lines = f.readlines()
    lines = list(filter(None, [re.sub('\n|[=>]|\'|"|,', '', line) for line in lines]))
decode_dict = {}
current_column = None
for line in lines:
    if line[0].isdigit():
        decode_dict[current_column].append(re.sub('^\d+\s+(.*\s+/\s+)?', '', line))
    else:
        current_column = line
        decode_dict[current_column] = []     

In [609]:
decoding_df = pd.DataFrame(dict([(k, pd.Series(v)) for k,v in decode_dict.items()]))
decoding_df.index += 1
decoding_df.head()

,results,versions,templates,colors,fractions,heroes
1,Draw,SoD,2sm4d(3),Red,Castle,Orrin
2,Win,WT,Balance,Blue,Rampart,Valeska
3,Lose,Hota,Panic,Green,Tower,Edric
4,NaN,NaN,Jebus Cross,Orange,Inferno,Sylvia
5,NaN,NaN,8mm6,Purple,Necropolis,Lord Haart


Using the just created DataFrame to decode the values:

In [610]:
deciphered_df = raw_df.copy()
deciphered_df['game_template'] = decoding_df.loc[deciphered_df['game_template'], 'templates'].values
deciphered_df['game_version'] = decoding_df.loc[deciphered_df['game_version'], 'versions'].values
deciphered_df['p1_fraction'] = decoding_df.loc[deciphered_df['p1_fraction'], 'fractions'].values
deciphered_df['p2_fraction'] = decoding_df.loc[deciphered_df['p2_fraction'], 'fractions'].values
deciphered_df['p1_hero_start'] = decoding_df.loc[deciphered_df['p1_hero_start'], 'heroes'].values
deciphered_df['p2_hero_start'] = decoding_df.loc[deciphered_df['p2_hero_start'], 'heroes'].values
deciphered_df['p_1_hero_main'] = decoding_df.loc[deciphered_df['p_1_hero_main'], 'heroes'].values
deciphered_df['p2_hero_main'] = decoding_df.loc[deciphered_df['p2_hero_main'], 'heroes'].values
deciphered_df['game_result'] = decoding_df.loc[deciphered_df['game_result'], 'results'].values
deciphered_df['p1'] = True
deciphered_df.dropna(inplace=True)
deciphered_df = deciphered_df[deciphered_df.game_version != 'SoD']
deciphered_df.reset_index(drop=True, inplace=True)

Fixing wrong match results:

In [611]:
players_ids = {"Drakonin":53651, "Dawidu91":28655, "Jaris":36344,
               "Jelen117":36750, "Szopa666":55968, "Tyranuxus" : 52903}

deciphered_df.loc[(deciphered_df['p2_id'] == deciphered_df['player'].map(players_ids))
                  & (deciphered_df['p2_rating_change'] > 0), ['game_result', 'p1']] = ['Win', False]
deciphered_df.loc[(deciphered_df['p2_id'] == deciphered_df['player'].map(players_ids))
                  & (deciphered_df['p2_rating_change'] < 0), ['game_result', 'p1']] = ['Lose', False]

### 3. Analysis

Ready-to-analyze dataframe:

In [612]:
deciphered_df.head()

,game_date,game_length,game_template,game_version,tournament_id,owner_id,p1_rating,p1_fraction,p1_hero_start,p_1_hero_main,p1_rating_change,p2_id,p2_rating,p2_hero_main,p2_hero_start,p2_fraction,p2_rating_change,game_result,player,p1
0,2018-06-07,236,6lm10a,Hota,109,44727,1532.0,Inferno,Marius,Isra,17.0,53651,1562.0,Gundula,Gundula,Stronghold,-17.0,Lose,Drakonin,False
1,2018-06-20,134,Diamond,Hota,109,53651,1541.0,Stronghold,Gundula,Gundula,16.0,49828,1544.0,Charna,Charna,Necropolis,-16.0,Win,Drakonin,True
2,2018-07-05,143,6lm10a,Hota,109,53651,1557.0,Stronghold,Gundula,Gundula,24.0,43065,1745.0,Luna,Luna,Conflux,-24.0,Win,Drakonin,True
3,2018-07-11,144,6lm10a,Hota,111,53651,1581.0,Stronghold,Dessa,Dessa,17.0,44871,1597.0,Thant,Thant,Necropolis,-17.0,Win,Drakonin,True
4,2018-08-10,131,6lm10a,Hota,109,53651,1598.0,Stronghold,Gundula,Crag Hack,13.0,49828,1513.0,Labetha,Charna,Necropolis,-13.0,Win,Drakonin,True
